# Dataloader
Hello dear NMRcrafter! In this notebook I would like to quickly run you through the usage of the nmrcraft dataloader!

### Initiate Dataloader
To initiate the dataloader you can either run the first cell or the second cell. The first cell will initiate the dataloader with the default parameters, whereas the second cell will initiate it with the parameters that you want to use. If you want to change the parameters of the dataloader, you can change them in the second cell!

Cell 1 here has the minimum args required to initalize the Dataloader.

In [24]:
from nmrcraft.data.dataset import DataLoader

feature_columns = [
            "M_sigma11_ppm",
            "M_sigma22_ppm",
            "M_sigma33_ppm",
            "E_sigma11_ppm",
            "E_sigma22_ppm",
            "E_sigma33_ppm",
        ]

data_loader = DataLoader(
    feature_columns=feature_columns,
    target_columns="metal",
)

### DataLoader Args
But of course there are a lot of args to choose from including these following that are meant to be chosen by you guys
- `target_columns`: should be defined via args but is that string that defines which targets are the ones you wanna let the model predict.
- `test_size`: defines test train split
- `random_state`: sets random state in dataloader, doesn't really need to be touched but if you for some reason want multiple dataloaders or whatever, go ahead.
- `dataset_size`: total size of dataset that is used. This is done directly at loading, so test train etc happens after this.
- `target_type`: defines whether you want the dataloader to be in 'one-hot' or 'categorical' mode.
- `complex_geometry`: defines what sort of complexes get loaded into the dataset. You can choose between oct, spy, tbp or all

In [25]:
from nmrcraft.data.dataset import DataLoader

feature_columns = [
            "M_sigma11_ppm",
            "M_sigma22_ppm",
            "M_sigma33_ppm",
            "E_sigma11_ppm",
            "E_sigma22_ppm",
            "E_sigma33_ppm",
        ]

data_loader = DataLoader(
    feature_columns=feature_columns,
    target_columns="metal_X4_E",
    test_size=0.2,
    random_state=42,
    dataset_size=0.001,
    target_type="categorical",  # can be "categorical" or "one-hot"
    complex_geometry="all", # can be oct, spy, tbp or all
)

### Load data
To load the dataset you can just call the `load_data()` methodhod. This will return the X and y tests and trains respectively and also the y_labels.

In [26]:
X_train, X_test, y_train, y_test, y_labels = data_loader.load_data()


### Categorical Features (X array for categorical mode)
In the example below the first six numbers are the scaled NMR features, so the tensors passed to the dataloader (without data leakage 🥳).
The following integers are the rest of the structural features that are not targets, so in this case as metal_X4_E are targets the integers correspond to X1_X2_X3_L (not sure about ordering but that shouldn't matter to the ML model)

In [27]:
print(X_test)

[[-1.97766652 -1.18731438 -2.2352046   0.3188357   0.31110165  0.32103837
   0.          0.          0.          2.        ]
 [-3.01966557 -1.68991936 -0.65836536  0.64940143  0.24379132  0.32555844
   0.          0.          0.          2.        ]
 [ 3.91914844 -1.3967671  -1.84610018  6.59283973 -0.70845918 -2.94953381
   1.          6.          1.          3.        ]
 [-0.73035351  0.19671936 -0.86673693 -0.24718666  0.25534291  0.40802954
   1.          1.          2.          3.        ]
 [-1.19117945 -0.90898261  0.14175844 -0.57344302  0.32122751  0.33177741
   1.          5.          1.          2.        ]
 [ 0.39324608  0.38777281  1.37421965 -0.46795372  0.31074452  0.24188147
   1.          1.          3.          2.        ]]


### Target Features (y array for categorical mode)
Here we have the columns representing metal, X4 and E. The integers come from the LabelEndoder and each integer corresponds to one Ligand/metal/thing. For example the first row 1, 1, 2 is 'W', 'Br', 'imido26diisopropylphenyl' (as can be seen in the decoder below).

In [28]:
print(y_test)

[[ 1  1  2]
 [ 0  1  4]
 [ 1  6 14]
 [ 1  9 11]
 [ 0  3 12]
 [ 0  6 11]]


### Decode target (y) arrays
To decode the targets you got you can use either the `categorical_target_decoder()` or the `binarized_target_decoder()` method of the dataloader. This will return an array where each position is the string corresponding to what it is. As far as I know these decoded things can be easily passed into the confusion matrix or maybe after flattening them.

__Keep in mind that the decoding is dataloader specific as you need the same encoder to decode your data so always use the same dataloader for the targets predicted from a X__

Basically: dataloader1 = DataLoader makes X1. Model is trained on X1 and y1 from dataloader1. the predictions of the model must be decoded with dataloader1 and you can't make another dataloader2 = DataLoader and try to decode the y1_pred with dataloader2. 

In [29]:
data_loader.categorical_target_decoder(y_test)


array([['W', 'Br', 'imido26diisopropylphenyl'],
       ['Mo', 'Br', 'imido35diisopropylphenyl'],
       ['W', 'pentafluorophenoxy', 'selenido'],
       ['W', 'thiophenoxy', 'imidotertbutyl'],
       ['Mo', 'ethyl', 'imidotrityl'],
       ['Mo', 'pentafluorophenoxy', 'imidotertbutyl']], dtype='<U24')

In [30]:
data_loader.confusion_matrix_data_adapter_categorical(y_test)

['W',
 'Br',
 'imido26diisopropylphenyl',
 'Mo',
 'Br',
 'imido35diisopropylphenyl',
 'W',
 'pentafluorophenoxy',
 'selenido',
 'W',
 'thiophenoxy',
 'imidotertbutyl',
 'Mo',
 'ethyl',
 'imidotrityl',
 'Mo',
 'pentafluorophenoxy',
 'imidotertbutyl']

### Confusion matrix with the dataloader
Here just an example how nice the sklearn funcitons seem to work with labeld data.

In [31]:
from sklearn.metrics import confusion_matrix
y_true_cm = y_pred_cm = data_loader.confusion_matrix_data_adapter_categorical(y_test)
confusion_matrix(y_true_cm, y_pred_cm)

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### One-hot
Here just the same stuff with one hot as an example so you see how the arrays look like. To be noted is that the structural features are one-hot encoded, so each integer column corresponds to a thing. [Here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) you can read up on the one hot encoder.

In [32]:
from nmrcraft.data.dataset import DataLoader

feature_columns = [
            "M_sigma11_ppm",
            "M_sigma22_ppm",
            "M_sigma33_ppm",
            "E_sigma11_ppm",
            "E_sigma22_ppm",
            "E_sigma33_ppm",
        ]

data_loader = DataLoader(
    feature_columns=feature_columns,
    target_columns="metal_X3",
    test_size=0.2,
    random_state=42,
    dataset_size=0.0003,
    target_type="one-hot",  # can be "categorical" or "one-hot"
    complex_geometry="all", # can be oct, spy, tbp or all
)

X_train, X_test, y_train, y_test, y_labels = data_loader.load_data()

print(X_test)

[[  1.18246403  -2.41755974  -1.18298361 -13.09288379  -5.00596047
  -18.84526524   1.           0.           1.           0.
    0.           0.           0.           0.           0.
    1.           1.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           1.        ]
 [ -0.73076559  -0.63232685  -0.25949934   1.65795174   0.133269
    0.26070737   1.           0.           1.           0.
    0.           0.           1.           0.           0.
    0.           0.           1.           0.           0.
    0.           0.           1.           0.           0.
    0.           0.        ]]


### Label Binarized target (y-array one-hot mode)
Here you can see that for example the 5th column of the array corresponds to tertbutoxy9F and is set to 1, so True and the fourth corresponds to phenoxy and is therefore set to true on column four.
The metal column is special as it was already binary so the metals only get a single column that is either True or False. So when the metal column is False it's molly and True means tungsten (in this particular case. This is automatically determined by the [LabelBinarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html#sklearn.preprocessing.LabelBinarizer)).

In [33]:
print(y_test)

[[0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]]


In [34]:
data_loader.binarized_target_decoder(y_test)

[['Mo', 'tertbutoxy9F'], ['Mo', 'phenoxy']]

In [35]:
data_loader.confusion_matrix_data_adapter_one_hot(y_test)

['Mo', 'tertbutoxy9F', 'Mo', 'phenoxy']